In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import *

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
#Creating a Dataframe
df=spark.read.load('/FileStore/tables/Airline_Dataset_Updated___v2-1.csv',format='csv',sep=',',header='true',escape='"', inferSchema='true')

In [0]:
df.count()

Out[48]: 98619

In [0]:
df.show(1)

+------------+----------+---------+------+---+-----------+----------------+--------------------+-------------+-----------------+-------------+--------------+---------------+-------------------+-------------+
|Passenger ID|First Name|Last Name|Gender|Age|Nationality|    Airport Name|Airport Country Code| Country Name|Airport Continent|   Continents|Departure Date|Arrival Airport|         Pilot Name|Flight Status|
+------------+----------+---------+------+---+-----------+----------------+--------------------+-------------+-----------------+-------------+--------------+---------------+-------------------+-------------+
|      ABVWIg|    Edithe|   Leggis|Female| 62|      Japan|Coldfoot Airport|                  US|United States|              NAM|North America|     6/28/2022|            CXF|Fransisco Hazeldine|      On Time|
+------------+----------+---------+------+---+-----------+----------------+--------------------+-------------+-----------------+-------------+--------------+-----------

In [0]:
df.printSchema()

root
 |-- Passenger ID: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Airport Name: string (nullable = true)
 |-- Airport Country Code: string (nullable = true)
 |-- Country Name: string (nullable = true)
 |-- Airport Continent: string (nullable = true)
 |-- Continents: string (nullable = true)
 |-- Departure Date: string (nullable = true)
 |-- Arrival Airport: string (nullable = true)
 |-- Pilot Name: string (nullable = true)
 |-- Flight Status: string (nullable = true)



# **Data Cleaning**

In [0]:
# dropping un necessory columns 
df=df.drop('First Name','Last Name','Passenger ID','Airport Country Code','Airport Continent')

In [0]:
# Rename columns
df = df.withColumnRenamed("Airport Name", "Airport_name") \
               .withColumnRenamed("Country Name", "Country_Name")\
                   .withColumnRenamed("Departure Date","Departure_Date")\
                       .withColumnRenamed("Flight Status","Flight_Status")\
                           .withColumnRenamed("Pilot Name","Pilot_Name")


In [0]:
df.printSchema()

root
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Nationality: string (nullable = true)
 |-- Airport_name: string (nullable = true)
 |-- Country_Name: string (nullable = true)
 |-- Continents: string (nullable = true)
 |-- Departure_Date: string (nullable = true)
 |-- Arrival Airport: string (nullable = true)
 |-- Pilot_Name: string (nullable = true)
 |-- Flight_Status: string (nullable = true)



In [0]:
# Creating sql Table for anlaysis
df.createOrReplaceTempView('airline')

In [0]:
%sql select * from airline
limit 4;

Gender,Age,Nationality,Airport_name,Country_Name,Continents,Departure_Date,Arrival Airport,Pilot Name,Flight_Status
Female,62,Japan,Coldfoot Airport,United States,North America,6/28/2022,CXF,Fransisco Hazeldine,On Time
Male,62,Nicaragua,Kugluktuk Airport,Canada,North America,12/26/2022,YCO,Marla Parsonage,On Time
Male,67,Russia,Grenoble-Isère Airport,France,Europe,1/18/2022,GNB,Rhonda Amber,On Time
Female,71,China,Ottawa / Gatineau Airport,Canada,North America,9/16/2022,YND,Kacie Commucci,Delayed


# **People gender wise**

In [0]:
%sql
select Gender, count(*) as Total
from airline
group by Gender;

Gender,Total
Female,49021
Male,49598


# **Lowest top 5 nationalities**

In [0]:
%sql
select Nationality,count(*) as people
from airline
group by Nationality
order by people limit 5;

Nationality,people
Jersey,1
Norfolk Island,1
Sint Maarten,1
French Southern Territories,2
Svalbard and Jan Mayen,2


# **Top 5 nationalities**

In [0]:
%sql
select Nationality, count(*) as total
from airline
group by Nationality
order by total desc limit 5;

Nationality,total
China,18317
Indonesia,10559
Russia,5693
Philippines,5239
Brazil,3791


# Total Airports

In [0]:
%sql
select count(distinct Airport_Name)
from airline;

count(DISTINCT Airport_Name)
9062


# Top 5 Airports with highest number of passengers

In [0]:
%sql
select Airport_name, count(*) as total
from airline
group by Airport_name
order by total desc
limit 5;

Airport_name,total
San Pedro Airport,43
Santa Maria Airport,38
Böblingen Flugfeld,36
Santa Ana Airport,35
San Fernando Airport,31


# Top 5 airports with lowest no. of passengers

In [0]:
%sql
select Airport_Name,count(*) as total
from airline
group by Airport_Name
order by total limit 5;

Airport_Name,total
Falcon State Airport,1
Hiroshima Airport,1
Juba International Airport,2
Balıkesir Merkez Airport,2
Mount Gambier Airport,2


# Flight Status

In [0]:
%sql
select * from airline
limit 5;

Gender,Age,Nationality,Airport_name,Country_Name,Continents,Departure_Date,Arrival Airport,Pilot Name,Flight_Status
Female,62,Japan,Coldfoot Airport,United States,North America,6/28/2022,CXF,Fransisco Hazeldine,On Time
Male,62,Nicaragua,Kugluktuk Airport,Canada,North America,12/26/2022,YCO,Marla Parsonage,On Time
Male,67,Russia,Grenoble-Isère Airport,France,Europe,1/18/2022,GNB,Rhonda Amber,On Time
Female,71,China,Ottawa / Gatineau Airport,Canada,North America,9/16/2022,YND,Kacie Commucci,Delayed
Male,21,China,Gillespie Field,United States,North America,2/25/2022,SEE,Ebonee Tree,On Time


In [0]:
%sql
select Flight_Status, count(*) as total
from airline
group by Flight_Status
order by total desc;

Flight_Status,total
Cancelled,32942
On Time,32846
Delayed,32831


# Flight Status continent wise

In [0]:
# Total On time Flights

In [0]:
%sql
SELECT 
    Continents,
    COUNT(*) AS total,
    COUNT(CASE WHEN Flight_Status = 'Delayed' THEN 1 END) AS delayed,
    COUNT(CASE WHEN Flight_Status = 'On Time' THEN 1 END) AS on_time,
    COUNT(CASE WHEN Flight_Status = 'Cancelled' THEN 1 END) AS canceled
FROM 
    airline
GROUP BY 
    Continents
ORDER BY 
    total DESC;


Continents,total,delayed,on_time,canceled
North America,32033,10696,10644,10693
Asia,18637,6160,6242,6235
Oceania,13866,4634,4613,4619
Europe,12335,4178,4062,4095
Africa,11030,3654,3719,3657
South America,10718,3509,3566,3643
